# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Aluno**: Eduardo Garcia do Nascimento

**RA/CPF**: 22008732800


---
### Carregamento dos dados

Nesta seção é feita a carga dos atributos em um dataframe só, ou seja, os três datasets são lidos e concatenados para que a redução de atributos leve em conta o que existe de melhor em todos eles.


In [1]:
# -*- coding: utf-8 -*-

# Caminho dos arquivos
FILES_DIRECTORY = "data"

import numpy as np 
import pandas as pd 
import os
from scripts import utils

if __name__ == '__main__':
    
                      
    # importa o arquivo e guarda em um dataframe do Pandas
    set1_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set1.csv'), sep=',', low_memory=False)
    set2_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set2.csv'), sep=',', low_memory=False) 
    set3_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set3.csv'), sep=',', low_memory=False)
    train_dataset = pd.read_csv(os.path.join(FILES_DIRECTORY, 'train.csv'), sep=',')
    test_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'test.csv'), sep=',')
    
    # Renomeia colunas concatenando o setX antes de fazer o merge para identificá-las posteriormente
    cols = set1_dataset.columns
    for col in cols:
        set1_dataset = set1_dataset.rename(columns={col:'set1_'+col})
        
    cols = set2_dataset.columns
    for col in cols:
        set2_dataset = set2_dataset.rename(columns={col:'set2_'+col})
    
    cols = set3_dataset.columns
    for col in cols:
        set3_dataset = set3_dataset.rename(columns={col:'set3_'+col})

    # Concatena os datasets em somente um dataset único
    frames = [ set3_dataset ]
    combined_dataset = pd.concat(frames, axis=1)
    
    
    set1_dataset['classe'] = np.nan
    set2_dataset['classe'] = np.nan
    set3_dataset['classe'] = np.nan
    
    
    set1_dataset.loc[train_dataset['Id'].values,'classe'] = train_dataset['Class'].values
    set2_dataset.loc[train_dataset['Id'].values,'classe'] = train_dataset['Class'].values
    set3_dataset.loc[train_dataset['Id'].values,'classe'] = train_dataset['Class'].values
    
    #samples_mask = (input_dataset.classe!=-1)&(input_dataset.classe!=1)&(input_dataset.classe!=0)
    #samples_df = input_dataset[samples_mask].drop('classe', axis=1)
    
    K1_df = set1_dataset.loc[test_dataset.iloc[:,:].values.T[0]].copy()
    K1_df = K1_df.drop('classe', axis=1)
    K2_df = set2_dataset.loc[test_dataset.iloc[:,:].values.T[0]].copy()
    K2_df = K2_df.drop('classe', axis=1)
    K3_df = set3_dataset.loc[test_dataset.iloc[:,:].values.T[0]].copy()
    K3_df = K3_df.drop('classe', axis=1)
    
    #print('A base de dados inicial combinada tem %d amostras com %d atributos.' % (input_dataset.shape[0],
    #                                                                             input_dataset.shape[1]))
    
        
   
    utils.beep(1, 400)

---
### Pré-processamento e seleção de atributos

Nesta seção são feitas limpezas da base de dados como:

* Remoção de atributos sem variância;
* Tratamento de outliers e dados nulos;
* Seleção dos atributos que terão maior valor para o algoritmo de classificação.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

X1_df = set1_dataset.loc[(set1_dataset['classe'] == -1) | (set1_dataset['classe'] == 1)]
X2_df = set2_dataset.loc[(set2_dataset['classe'] == -1) | (set2_dataset['classe'] == 1)]
X3_df = set3_dataset.loc[(set3_dataset['classe'] == -1) | (set3_dataset['classe'] == 1)]

print('Separando a base em treino e teste')
X1_train_df, X1_test_df, y1_train_df, y1_test_df = train_test_split(X1_df.drop('classe', axis=1), 
                                                    X1_df[['classe']], 
                                                    test_size=0.2, 
                                                    stratify=X1_df[['classe']],
                                                    random_state=0, 
                                                    )
X2_train_df, X2_test_df, y2_train_df, y2_test_df = train_test_split(X2_df.drop('classe', axis=1), 
                                                    X2_df[['classe']], 
                                                    test_size=0.2, 
                                                    stratify=X2_df[['classe']],
                                                    random_state=0, 
                                                    )
X3_train_df, X3_test_df, y3_train_df, y3_test_df = train_test_split(X3_df.drop('classe', axis=1), 
                                                    X3_df[['classe']], 
                                                    test_size=0.2, 
                                                    stratify=X3_df[['classe']],
                                                    random_state=0, 
                                                    )

y_df = X1_df[['classe']].copy()
X1_df = X1_df.drop('classe', axis=1)
X2_df = X2_df.drop('classe', axis=1)
X3_df = X3_df.drop('classe', axis=1)

# Remove os atributos que são constantes e não oferecem nenhum valor aos algoritmos de classificação
variance_mask = VarianceThreshold().fit(X1_train_df).get_support()
X1_train_df = X1_train_df.iloc[:,variance_mask]
X1_test_df = X1_test_df.iloc[:,variance_mask]
print('Atributos removidos por baixa variância: %d' % np.sum(~variance_mask))

variance_mask = VarianceThreshold().fit(X2_train_df).get_support()
X2_train_df = X2_train_df.iloc[:,variance_mask]
X2_test_df = X2_test_df.iloc[:,variance_mask]
print('Atributos removidos por baixa variância: %d' % np.sum(~variance_mask))

variance_mask = VarianceThreshold().fit(X3_train_df).get_support()
X3_train_df = X3_train_df.iloc[:,variance_mask]
X3_test_df = X3_test_df.iloc[:,variance_mask]
print('Atributos removidos por baixa variância: %d' % np.sum(~variance_mask))

variance_mask = VarianceThreshold().fit(X1_df).get_support()
X1_df = X1_df.iloc[:,variance_mask]
K1_df = K1_df.iloc[:,variance_mask]
variance_mask = VarianceThreshold().fit(X2_df).get_support()
X2_df = X2_df.iloc[:,variance_mask]
K2_df = K2_df.iloc[:,variance_mask]
variance_mask = VarianceThreshold().fit(X3_df).get_support()
X3_df = X3_df.iloc[:,variance_mask]
K3_df = K3_df.iloc[:,variance_mask]

# Tratamento de outliers e entradas nulas
outliers_df = X1_train_df.copy()
imp_mean = IterativeImputer(max_iter=10, random_state=0)
imp_mean = imp_mean.fit(outliers_df)
X1_train_df.loc[:,:] = imp_mean.transform(X1_train_df)
X1_test_df.loc[:,:] = imp_mean.transform(X1_test_df)
outliers_df = X2_train_df.copy()
imp_mean = IterativeImputer(max_iter=10, random_state=0)
imp_mean = imp_mean.fit(outliers_df)
X2_train_df.loc[:,:] = imp_mean.transform(X2_train_df)
X2_test_df.loc[:,:] = imp_mean.transform(X2_test_df)
outliers_df = X3_train_df.copy()
imp_mean = IterativeImputer(max_iter=10, random_state=0)
imp_mean = imp_mean.fit(outliers_df)
X3_train_df.loc[:,:] = imp_mean.transform(X3_train_df)
X3_test_df.loc[:,:] = imp_mean.transform(X3_test_df)


outliers_df = X1_df.copy()
imp_mean = IterativeImputer(max_iter=10, random_state=0)
imp_mean = imp_mean.fit(outliers_df)
X1_df.loc[:,:] = imp_mean.transform(X1_df)
K1_df.loc[:,:] = imp_mean.transform(K1_df)
outliers_df = X2_df.copy()
imp_mean = IterativeImputer(max_iter=10, random_state=0)
imp_mean = imp_mean.fit(outliers_df)
X2_df.loc[:,:] = imp_mean.transform(X2_df)
K2_df.loc[:,:] = imp_mean.transform(K2_df)
outliers_df = X3_df.copy()
imp_mean = IterativeImputer(max_iter=10, random_state=0)
imp_mean = imp_mean.fit(outliers_df)
X3_df.loc[:,:] = imp_mean.transform(X3_df)
K3_df.loc[:,:] = imp_mean.transform(K3_df)



# Normalização dos dados entre 0 e 1
scaler = MinMaxScaler().fit(X1_train_df)
X1_train_df.loc[:,:] = scaler.transform(X1_train_df)
X1_test_df.loc[:,:] = scaler.transform(X1_test_df)
scaler = MinMaxScaler().fit(X2_train_df)
X2_train_df.loc[:,:] = scaler.transform(X2_train_df)
X2_test_df.loc[:,:] = scaler.transform(X2_test_df)
scaler = MinMaxScaler().fit(X3_train_df)
X3_train_df.loc[:,:] = scaler.transform(X3_train_df)
X3_test_df.loc[:,:] = scaler.transform(X3_test_df)


scaler = MinMaxScaler().fit(X1_df)
X1_df.loc[:,:] = scaler.transform(X1_df)
K1_df.loc[:,:] = scaler.transform(K1_df)
scaler = MinMaxScaler().fit(X2_df)
X2_df.loc[:,:] = scaler.transform(X2_df)
K2_df.loc[:,:] = scaler.transform(K2_df)
scaler = MinMaxScaler().fit(X3_df)
X3_df.loc[:,:] = scaler.transform(X3_df)
K3_df.loc[:,:] = scaler.transform(K3_df)




'''
[False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False False False  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True False
  True False False  True  True False  True  True  True False  True  True
  True  True  True  True  True  True  True False  True  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False  True  True  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True
  True  True False  True  True  True  True  True False  True  True  True
 False  True  True  True  True  True  True False False  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True
  True False False False False False False False  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
 False  True  True False  True  True  True  True  True  True  True  True
  True False  True  True  True  True False  True False  True  True  True
  True  True  True]
[32 20  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 21
  1  1  1  1  1  1  7  1  1  1  1  1  1  1  1  1  1  1 44  1  1  1  1  1
 31  1  1  1  1  1  1  1  1  1  1  1 18  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1 27  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  2  1 24  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1 19 40 34  1  1 36  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1 41  1  1  1  1  1  1  1  1  1 26  1 10 11  1  1  5  1  1  1 39  1  1
  1  1  1  1  1  1  1 30  1  1 17  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  3  4  1  1  1  1  1  1  1  1  1  1  1 38  1  1  1  1  1  1
  1  1 13  1  1  1  1  1 29  1  1  1 43  1  1  1  1  1  1 25 33  1  1  1
  1  1  1 35  1  1  1  1  1  1  1  1  1 23 22  9  8 42 16 15  1  1  1  1
  1  1 14  1  1  1  1  1  1  1  1  1  6  1  1 28  1  1  1  1  1  1  1  1
  1 45  1  1  1  1 37  1 12  1  1  1  1  1  1]
'''


#print('\n\n\n\nBreve avaliação das primeiras amostras')
#display(X_train_df.head(10))

#print('Avaliação do descritivo do dataset que permite ter uma ideia mais realista dos dados')
#display(X_train_df.describe())

utils.beep(1, 500)

Separando a base em treino e teste
Atributos removidos por baixa variância: 3
Atributos removidos por baixa variância: 2
Atributos removidos por baixa variância: 1


---
### Análise exploratória

Nesta seção são exibidas informações do resultado após os dados serem pré-processados e os atributos selecionados. Dentre as ferramentas para análise exploratória que serão utilizados estão:

* Descritivo resumido da base.
* Análises de covariância e correlação.
* Matriz de disperação entre todos os atributos selecionados. 
* Diagramas de violino para visualização dos quartis e outliers como uma variação aos diagramas de caixa.
* Gráfico de dispersão com a dimensionalidade reduzida a somente 2 atributos.

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from scripts import analise_exploratoria

exploratory_analisys = False

if exploratory_analisys == True:
    # Análise do balanceamento das classes
    print('Análise e visualização dos dados:')
    ax = sns.countplot(x='classe', data=pd.concat([X_train_df,y_train_df], axis=1), label="Contagem")

    N,P = y_train_df.classe.value_counts()
    print('Número de posts comuns:', N)
    print('Número de posts phishing:', P)
    plt.show()

    # Análise da matriz scatter pra que entendamos a relação entre os atributos
    print('Análise da matriz de dispersão')
    sns.pairplot(input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)], hue='classe', height=3.5);
    plt.show()

    # matrizes de covariancia e correlação
    print('Análise das matrizes de covariância ')
    df_covariance = X_train_df.iloc[:,:-1].cov()
    df_correlation = X_train_df.iloc[:,:-1].corr()
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20, 8))
    plt.title('Covariância')
    sns.heatmap(df_covariance, annot=True, xticklabels=df_correlation.columns, 
                yticklabels=df_correlation.columns, ax=ax1)
    plt.title('Correlação')
    sns.heatmap(df_correlation, annot=True, xticklabels=df_correlation.columns, 
                yticklabels=df_correlation.columns, ax=ax2)
    plt.show()


    # Diagramas de caixa
    plt.figure(figsize=(20,10))
    data = pd.melt(pd.concat([X_train_df,y_train_df], axis=1),
                   id_vars="classe", var_name="features", value_name='value')
    sns.boxplot(x='features', y='value', hue='classe', data=data)
    plt.show()

    # Diagramas de violino
    plt.figure(figsize=(20,10))
    sns.violinplot(x='features', y='value', hue='classe', data=data, split=True, inner="quartile")
    plt.show()

    # Separação de atributos e classe para 
    analise_exploratoria.printPCA(X_train_df.values,y_train_df.values.T[0])    

    analise_exploratoria.printJointPlot(X_train_df,y_train_df)    

utils.beep(1, 600)   

---
### Preparo dos dados e experimentos para encontrar os melhores hiperparâmetros

Nesta seção os dados são separados em duas partes: treino e testes. Esta estratégia foi utilizada para posterior comparação com a validação de modelos utilizandos K-folds.
Para encontrar os melhores hiperparâmetros foi utilizada a classe GridSearchCV e devido o seu alto custo computacional e não existência da necessidade de executá-las sempre, a sua chamada é condicionada às variáveis booleanas evaluate_svm_hiperparameters, evaluate_rfc_hiperparameters e evaluate_lrc_hiperparameters serem verdadeiras.

In [4]:
from scripts import preprocessamento
from scripts import experimentos
from scripts import utils

X1_train = X1_train_df.values
y1_train = y1_train_df.values.T[0]
X1_test = X1_test_df.values
y1_test = y1_test_df.values.T[0]
X2_train = X2_train_df.values
y2_train = y2_train_df.values.T[0]
X2_test = X2_test_df.values
y2_test = y2_test_df.values.T[0]
X3_train = X3_train_df.values
y3_train = y3_train_df.values.T[0]
X3_test = X3_test_df.values
y3_test = y3_test_df.values.T[0]

X1 = X1_df.values
X2 = X2_df.values
X3 = X3_df.values
y = y_df.values.T[0]

# Utilize as flags abaixo somente para avaliação dos hiperparâmetros pois elas demoram muito pra serem executadas
evaluate_svm_hiperparameters = False # Busca os melhores parâmetros para as máquinas de vetores de suporte
evaluate_rfc_hiperparameters = False # Busca os melhores parâmetros para as florestas aleatórias
evaluate_knn_hiperparameters = False # Busca os melhores parâmetros para o KNN
evaluate_mlp_hiperparameters = False # Busca os melhores parâmetros para o KNN

print('Removendo amostras outliers - Removido pois estava piorando os resultados')
#X_train, y_train = preprocessamento.remove_outliers(X_train, y_train)
#X,y = preprocessamento.remove_outliers(X, y)

scores = [ 
        #    'balanced_accuracy', 
        #    'f1', 
            'roc_auc'
        ]

if evaluate_svm_hiperparameters == True:
    experimentos.find_best_svm(X,y, scores)
    
if evaluate_rfc_hiperparameters == True:    
    experimentos.find_best_rfc(X, y, scores)
        
if evaluate_knn_hiperparameters == True:
    experimentos.find_best_knn(X, y, scores)

if evaluate_mlp_hiperparameters == True:
    experimentos.find_best_mlp(X, y, scores)
    

utils.beep(1, 700)

Removendo amostras outliers - Removido pois estava piorando os resultados


---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais ensinados no curso e executando os métodos inteligentes

In [15]:
# Importa os classificadores usados nesse trabalho
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm

from scripts import experimentos
from scripts import preprocessamento

collect_more_samples = False
collect_more_samples_again = False

X1_train_bal, y1_train_bal = X1_train, y1_train
X2_train_bal, y2_train_bal = X2_train, y2_train
X3_train_bal, y3_train_bal = X3_train, y3_train

scoring=['roc_auc', 'f1', 'f1_micro', 'f1_macro', 'f1_weighted', 'accuracy', 'balanced_accuracy', 'precision', 'recall']

results = pd.DataFrame()

model_list = [
#    ['SVM poly', svm.SVC(kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
#    ['SVM linear', svm.SVC(kernel='linear', C=100, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
#    ['SVM rbf', svm.SVC(kernel='rbf', C=1000, gamma=0.0001, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
#    ['SVM rbf RA 50', svm.SVC(kernel='rbf', C=50, gamma='scale', class_weight='balanced', probability=True,random_state=1), 0],
#    ['SVM rbf RA 20', svm.SVC(kernel='rbf', C=20, gamma='scale', class_weight='balanced', probability=True,random_state=1), 0],
    #['SVM rbf RA', svm.SVC(kernel='rbf', C=10, gamma='scale', class_weight='balanced', probability=True,random_state=1), 0],
#    ['SVM Optimal', svm.SVC(kernel='rbf', C=1, gamma='scale', class_weight='balanced', probability=True,random_state=1), 0],
#    ['Random Forest', RandomForestClassifier(max_depth=2, class_weight='balanced', random_state=1), 0],
#    ['Random Forest Optimal', RandomForestClassifier(criterion='gini', max_depth=6, max_features='auto', n_estimators=500, class_weight='balanced', random_state=1), 0],
#    ['Logistic Regression', LogisticRegression(random_state=1, class_weight='balanced', max_iter=15000), 0],
#    ['Multinomial NB', MultinomialNB(), 0],
#     ['MLP', MLPClassifier( alpha=1e-5, hidden_layer_sizes=(600,), random_state=1, max_iter=5000, ), 0],
     ['MLP7', MLPClassifier( alpha=1e-5, hidden_layer_sizes=(700,), random_state=1, max_iter=5000, ), 0],
#    ['MLP rec3', MLPClassifier( alpha=1e-5, hidden_layer_sizes=(600,40,), random_state=1, max_iter=5000, ), 0],
    #['MLP Optimal', MLPClassifier( alpha=1e-5, hidden_layer_sizes=(600,), random_state=1, max_iter=5000, ), 0],
#    ['KNN', KNeighborsClassifier(n_neighbors=1, leaf_size=4, weights='distance'), 0],  
#    ['KNN Optimal', KNeighborsClassifier(algorithm='auto', n_neighbors=29, leaf_size=1, weights='distance'), 0],
#    ['AdaBoost', AdaBoostClassifier(n_estimators=100, random_state=0), 0],
#    ['GradBoost', GradientBoostingClassifier(random_state=0), 0],
]

# Pré balanceamento dos dados utilizando a técnica de oversampling
X1_train_bal, y1_train_bal = preprocessamento.oversample(X1_train_bal, y1_train_bal, times=3)
X1_bal, y_bal = preprocessamento.oversample(X1, y, times=3)
X2_train_bal, y2_train_bal = preprocessamento.oversample(X2_train_bal, y2_train_bal, times=3)
X2_bal, y_bal = preprocessamento.oversample(X2, y, times=3)
X3_train_bal, y3_train_bal = preprocessamento.oversample(X3_train_bal, y3_train_bal, times=3)
X3_bal, y_bal = preprocessamento.oversample(X3, y, times=3)

# População dos dados não classificados com o melhor classificador encontrado com os resultados mais confiáveis
if collect_more_samples == True:
    semi_model = svm.SVC(kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1)
    
    X_semi, y_semi, proba_mask = preprocessamento.add_samples(semi_model, X_bal, y_bal, samples_df.values)

    print('Número de amostras acrescentadas ao dataset de treinamento:', X_semi.shape[0], 
                                                                          np.sum(y_semi==-1), np.sum(y_semi==1))
    
    pos_mask = y_semi==1
    X_semi_positives = X_semi[pos_mask]
    y_semi_positives = y_semi[pos_mask]
    
    X_semi_positives = X_semi_positives
    y_semi_positives = y_semi_positives
    
    
    X_train_bal = np.concatenate([X_train_bal, X_semi_positives], axis=0)
    y_train_bal = np.concatenate([y_train_bal, y_semi_positives], axis=0)
    X_bal = np.concatenate([X_bal, X_semi_positives], axis=0)
    y_bal = np.concatenate([y_bal, y_semi_positives], axis=0)

if collect_more_samples_again == True:
    X_train_bal, y_train_bal = preprocessamento.oversample(X_train_bal, y_train_bal)
    X_bal, y_bal = preprocessamento.oversample(X_bal, y_bal)

    # População dos dados não classificados com o melhor classificador encontrado com os resultados mais confiáveis
    samples = samples[~proba_mask]
    X_semi, y_semi, proba_mask = preprocessamento.add_samples(semi_model, X_train_bal, y_train_bal, samples)
    print('Número de amostras acrescentadas ao dataset de treinamento:', X_semi.shape[0], 
                                                                          np.sum(y_semi==-1), np.sum(y_semi==1))
    X_train_bal = np.concatenate([X_train_bal, X_semi], axis=0)
    y_train_bal = np.concatenate([y_train_bal, y_semi], axis=0)
    X_bal = np.concatenate([X_bal, X_semi], axis=0)
    y_bal = np.concatenate([y_bal, y_semi], axis=0)

#print(X_bal.shape, X_train_bal.shape, np.sum(y_bal==-1),np.sum(y_bal==1), np.sum(y_train_bal==-1),np.sum(y_train_bal==1))

for model in model_list:
    print(model[0] + '------------------------------------------------------------------------------------')
    #scores = experimentos.evaluate_model(model[1], X_train_bal, y_train_bal, X_test, y_test)
    scores = experimentos.cross_classifier(model[1], X1_train_bal, y1_train_bal, X1_test, y1_test,
                                                     X2_train_bal, y2_train_bal, X2_test, y2_test,
                                                     X3_train_bal, y3_train_bal, X3_test, y3_test,
                                          )
    for score in scores:
        results.loc[model[0], score] = np.mean(scores[score])
    


utils.beep(1, 800)

Balaceamento antes da SOBREamostragem 3230 218
Balaceamento após da SOBREamostragem 3230 654
Balaceamento antes da SOBREamostragem 4038 272
Balaceamento após da SOBREamostragem 4038 816
Balaceamento antes da SOBREamostragem 3230 218
Balaceamento após da SOBREamostragem 3230 654
Balaceamento antes da SOBREamostragem 4038 272
Balaceamento após da SOBREamostragem 4038 816
Balaceamento antes da SOBREamostragem 3230 218
Balaceamento após da SOBREamostragem 3230 654
Balaceamento antes da SOBREamostragem 4038 272
Balaceamento após da SOBREamostragem 4038 816
MLP7------------------------------------------------------------------------------------


---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos e comparados, através de tabelas e gráficos

In [16]:
from scripts import analise_resultados

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    
    return ['background-color: yellow' if v else '' for v in is_max]

pd.set_option('precision', 4)
results_highlited = results.style.apply(highlight_max)
display(results_highlited)

#best_model.fit(X,y)
#y_pred = best_model.predict(X_test)
#analise_resultados.print_rocauc_curve(y_test, y_pred)
#analise_resultados.plot_decision_boundaries(X_train, y_train, svm.SVC, kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1)


utils.beep(3, 1000)

,bal_acc_test,f1_weighted,f1_micro,precision,recall,mcc,roc_auc,TP,TN
MLP7,0.6432,0.9255,0.9304,0.4250,0.3148,0.3299,0.6573,785.0000,17.0000


---
### Criação do arquivo de sumissão no Kaggle

Na etapa final, o arquivo submission.csv é criado para ser enviado ao Kaggle de acordo com os padrões pré-definidos pela proposta do desafio.

In [17]:
submit_preference_score = 'MLP7'

for model in model_list:
    if model[0]==submit_preference_score:
        best_model = model[1]
        
# Montagem do dataset de teste para envio para o Kaggle


print('Criando arquivo de submissão para o modelo: ' + submit_preference_score)    
    
print('Imprimindo arquivo submission.csv ...')
clf1= best_model.fit(X1_bal, y_bal)
y1_proba = clf1.predict_proba(K1_df)
clf2= best_model.fit(X2_bal, y_bal)
y2_proba = clf2.predict_proba(K2_df)
clf3= best_model.fit(X3_bal, y_bal)
y3_proba = clf3.predict_proba(K3_df)

y_proba = np.empty(y1_proba.shape)


for i in range(y1_proba.shape[0]):
    if( (abs(y1_proba[i][0]-y1_proba[i][1]) >= abs(y2_proba[i][0]-y2_proba[i][1])) and (abs(y1_proba[i][0]-y1_proba[i][1]) >= abs(y3_proba[i][0]-y3_proba[i][1])) ):
        y_proba[i] = y1_proba[i]
    if( (abs(y2_proba[i][0]-y2_proba[i][1]) >= abs(y1_proba[i][0]-y1_proba[i][1])) and (abs(y2_proba[i][0]-y2_proba[i][1]) >= abs(y3_proba[i][0]-y3_proba[i][1])) ):
        y_proba[i] = y2_proba[i]
    if( (abs(y3_proba[i][0]-y3_proba[i][1]) >= abs(y2_proba[i][0]-y2_proba[i][1])) and (abs(y3_proba[i][0]-y3_proba[i][1]) >= abs(y1_proba[i][0]-y1_proba[i][1])) ):
        y_proba[i] = y3_proba[i]


y_pred_submission = y_proba[:,1]


result = np.zeros((K1_df.shape[0],2))
for i in range(K1_df.shape[0]):
    result[i][0] = test_dataset.iloc[:,:].values.T[0][i]
    result[i][1] = y_pred_submission[i]
resultdf = pd.DataFrame(data=result, columns=["Id", "Predicted"])
resultdf['Id'] = resultdf['Id'].astype(int)
resultdf['Predicted'] = resultdf['Predicted'].round(decimals=5)
resultdf.to_csv('submission.csv', index=False, float_format='%.5f')
print('####################################################################################################')

utils.beep(5, 200)

Criando arquivo de submissão para o modelo: MLP7
Imprimindo arquivo submission.csv ...
####################################################################################################
